# Main experiment

## imports and setup

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

from src.pgr_experiment import *
from src.utils import choose_indices
from src.config import MODELS_LIST
from src.math_dataset import load_questions

In [6]:
train_ds = load_questions("train", limit=200)
test_ds = load_questions("test", limit=200)
indices = choose_indices(len(train_ds), 3, seed=42)

## Compare Performance of each model on MATH dataset

In [12]:
results = {}
for model in MODELS_LIST:
    print(f"Running experiments for model: {model}")
    score = await ask_with_gold(model, train_ds, test_ds, indices)
    results[model] = score
print(results)

Running experiments for model: gpt-5-mini
Running experiments for model: o3-mini
Running experiments for model: gpt-4.1-mini
Running experiments for model: gpt-4o-mini
Running experiments for model: gpt-4.1-nano


{'gpt-5-mini': 0.95, 'o3-mini': 0.83, 'gpt-4.1-mini': 0.885, 'gpt-4o-mini': 0.25, 'gpt-4.1-nano': 0.765}


In [ ]:
# save results
with open("results/model_math_performance.json", "w") as f:
    json.dump(results, f)

In [13]:
pgr = await run_pgr_experiment(
    "gpt-4o-mini",
    "gpt-4.1-mini",
    train_ds,
    test_ds,
    indices,
    verbose=True
)
print("====================================")
print(f"PGR           : {pgr:.3f}")
print("====================================")


Acc (strong model with weak labels): 0.8650
Acc (strong model with gold labels): 0.8850
Acc (weak model with gold labels): 0.2900
PGR           : 0.966
